## 21. Zephyr 7B Beta

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1,2,3'
os.environ.get('CUDA_VISIBLE_DEVICES')

'1,2,3'

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, MistralForCausalLM, LlamaTokenizerFast
import torch


model_path = 'HuggingFaceH4/zephyr-7b-beta'
model = MistralForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16, device_map="auto")
tokenizer = LlamaTokenizerFast.from_pretrained(model_path) 

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [3]:
generation_config = {
    "max_new_tokens": 256,
    #do_sample: True,
    #top_k: 40,
}

In [4]:

# Contexto e questão
context = r"""
"""

question = """
Crie o código em Python de uma função para calcular a sequência de Fibonacci. 
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"


# Tokenizar o texto
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)

with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        **generation_config
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)

/var/ontologia/Servicos/ACS_LLMS/server/anaconda3/lib/python3.11/site-packages/transformers/generation/utils.py:1554: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(





Q: 
Crie o código em Python de uma função para calcular a sequência de Fibonacci. 

A:

```python
def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)
```

Explicação:

A função `fibonacci` recebe um número `n` como parâmetro e retorna o `n`-ésimo número da sequência de Fibonacci.

A sequência de Fibonacci começa com os números 0 e 1, e cada número subsequente é a soma dos dois anteriores.

Neste código, a função verifica se o número `n` é menor ou igual a 1. Se for, retorna o próprio número, pois os primeiros dois números da sequência são 0 e 1.

Caso contrário, a função chama recursivamente as funções `fibonacci(n-1)` e `fibonacci(n-2)` e soma os resultados.

Essa é a man


In [6]:
def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)

fibonacci(2)

1

<hr>

############################################  COMPREENSÃO E GERAÇÃO DE TEXTO  ############################################
############################################  AVALIAÇÃO DA SEMÂNTICA E SINTAXE EM PT-BR  ############################################

In [2]:
import gradio as gr

context = """Os investimentos em renda fixa são considerados uma das opções mais seguras do mercado financeiro e são especialmente atraentes para investidores com perfil conservador. Esses investimentos são compostos principalmente por títulos de dívida, seja pública ou privada. No Brasil, muitos bancos, incluindo o Banco do Brasil, oferecem uma variedade de opções de investimento em renda fixa. Esses investimentos têm como característica principal o retorno previsível, e o rendimento, na maioria das vezes, é acordado no momento da aplicação.
- **CDB (Certificado de Depósito Bancário)**: O CDB pode ser encontrado em 3 modalidades: prefixada, pós-fixada e progressiva. Enquanto o CDB prefixado tem um rendimento acordado no ato da aplicação, o pós-fixado e o progressivo são atrelados ao CDI, com o último tendo sua taxa definida com base no tempo de permanência da aplicação. Apesar do prazo de vencimento, o CDB pós-fixado e o progressivo podem ser resgatados a qualquer momento. É adequado para investidores que buscam mais rentabilidade do que a poupança e estão dispostos a abrir mão da liquidez diária. 
- **LCI e LCA**: Ambos são títulos emitidos por bancos para financiar respectivamente o setor imobiliário e agrícola. São ideais para investidores que buscam isenção de Imposto de Renda e desejam investir em setores específicos da economia, como imobiliário ou agrícola. 
- **Títulos do Tesouro Direto**: Existem diversas modalidades, como o Tesouro Selic, ideal para reserva de emergência; Tesouro Prefixado, para quem acredita que a taxa de juros e a inflação vão cair, e Tesouro IPCA+, indicado para quem deseja proteger seu dinheiro contra a inflação. Recomendados para quem busca diversificar sua carteira de investimentos com opções seguras. São títulos públicos emitidos pelo governo federal e são considerados os investimentos mais seguros do país devido à garantia do governo federal. 
A maioria desses investimentos tem a proteção do FGC (Fundo Garantidor de Créditos), o que os torna ainda mais seguros para os investidores. Além disso, o Banco do Brasil, uma das principais instituições financeiras do país, oferece todas essas opções de investimento com diferentes vantagens e condições, atendendo a diversos perfis de investidores, desde os mais conservadores até os mais arrojados."""

def answer_question(question):
    prompt = context + "\n\nPergunta: " + question + "\nResposta:"
    inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
    input_ids = inputs["input_ids"].to('cuda')  # Comente esta linha se não estiver usando GPU
    attention_mask = inputs["attention_mask"].to('cuda')  # Comente esta linha se não estiver usando GPU

    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=1024,
            no_repeat_ngram_size=2,
            early_stopping=True
        )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

interface = gr.Interface(fn=answer_question,
                         inputs=gr.Textbox(lines=5, label="Digite sua pergunta aqui"),
                         outputs="text",
                         title="Compreensão de texto sobre Investimentos em Renda Fixa",
                         description="Forneça uma pergunta para obter uma resposta baseada no contexto de investimentos em renda fixa.")

interface.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\marce\anaconda3\Lib\site-packages\gradio\queueing.py", line 495, in call_prediction
    output = await route_utils.call_process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\marce\anaconda3\Lib\site-packages\gradio\route_utils.py", line 231, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\marce\anaconda3\Lib\site-packages\gradio\blocks.py", line 1594, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\marce\anaconda3\Lib\site-packages\gradio\blocks.py", line 1176, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\marce\anaconda3\Lib\site-packages\anyio\to_thread.py", line 28, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(func, *args, cancellable=cancellable,
    

In [5]:
################# Question Answering: Compreensão de texto e xtração de conteúdo com Contexto de Domínio #################

# Contexto e questão
context = """
Os investimentos em renda fixa são considerados uma das opções mais seguras do mercado financeiro e são especialmente atraentes para investidores com perfil conservador. Esses investimentos são compostos principalmente por títulos de dívida, seja pública ou privada. No Brasil, muitos bancos, incluindo o Banco do Brasil, oferecem uma variedade de opções de investimento em renda fixa. Esses investimentos têm como característica principal o retorno previsível, e o rendimento, na maioria das vezes, é acordado no momento da aplicação.
- **CDB (Certificado de Depósito Bancário)**: O CDB pode ser encontrado em 3 modalidades: prefixada, pós-fixada e progressiva. Enquanto o CDB prefixado tem um rendimento acordado no ato da aplicação, o pós-fixado e o progressivo são atrelados ao CDI, com o último tendo sua taxa definida com base no tempo de permanência da aplicação. Apesar do prazo de vencimento, o CDB pós-fixado e o progressivo podem ser resgatados a qualquer momento. É adequado para investidores que buscam mais rentabilidade do que a poupança e estão dispostos a abrir mão da liquidez diária. 
- **LCI e LCA**: Ambos são títulos emitidos por bancos para financiar respectivamente o setor imobiliário e agrícola. São ideais para investidores que buscam isenção de Imposto de Renda e desejam investir em setores específicos da economia, como imobiliário ou agrícola. 
- **Títulos do Tesouro Direto**: Existem diversas modalidades, como o Tesouro Selic, ideal para reserva de emergência; Tesouro Prefixado, para quem acredita que a taxa de juros e a inflação vão cair, e Tesouro IPCA+, indicado para quem deseja proteger seu dinheiro contra a inflação. Recomendados para quem busca diversificar sua carteira de investimentos com opções seguras. São títulos públicos emitidos pelo governo federal e são considerados os investimentos mais seguros do país devido à garantia do governo federal. 
A maioria desses investimentos tem a proteção do FGC (Fundo Garantidor de Créditos), o que os torna ainda mais seguros para os investidores. Além disso, o Banco do Brasil, uma das principais instituições financeiras do país, oferece todas essas opções de investimento com diferentes vantagens e condições, atendendo a diversos perfis de investidores, desde os mais conservadores até os mais arrojados.

Pergunta: Com base nos seus conhecimentos e no texto, o que têm em comum LCI e LCA?
Resposta: Ambos são títulos emitidos por bancos para financiar o setor imobiliário e agrícola, ideais para investidores que buscam isenção de Imposto de Renda e desejam investir em setores específicos da economia.
"""



question = """
Com base nos seus conhecimentos e no texto apresentado sobre investimentos em renda fixa, explique qual é o investimento mais adequado para alguém que busca proteção contra a inflação.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenizar o texto
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=160,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Os investimentos em renda fixa são considerados uma das opções mais seguras do mercado financeiro e são especialmente atraentes para investidores com perfil conservador. Esses investimentos são compostos principalmente por títulos de dívida, seja pública ou privada. No Brasil, muitos bancos, incluindo o Banco do Brasil, oferecem uma variedade de opções de investimento em renda fixa. Esses investimentos têm como característica principal o retorno previsível, e o rendimento, na maioria das vezes, é acordado no momento da aplicação.
- **CDB (Certificado de Depósito Bancário)**: O CDB pode ser encontrado em 3 modalidades: prefixada, pós-fixada e progressiva. Enquanto o CDB prefixado tem um rendimento acordado no ato da aplicação, o pós-fixado e o progressivo são atrelados ao CDI, com o último tendo sua taxa definida com base no tempo de permanência da aplicação. Apesar do prazo de vencimento, o CDB pós-fixado e o progressivo podem ser resgatados a qualquer momento. É adequado para investi

In [12]:
################# Question Answering com Contexto de Domínio #################

# Contexto e questão
context = """
Os investimentos em renda fixa são considerados uma das opções mais seguras do mercado financeiro e são especialmente atraentes para investidores com perfil conservador. Esses investimentos são compostos principalmente por títulos de dívida, seja pública ou privada. No Brasil, muitos bancos, incluindo o Banco do Brasil, oferecem uma variedade de opções de investimento em renda fixa. Esses investimentos têm como característica principal o retorno previsível, e o rendimento, na maioria das vezes, é acordado no momento da aplicação.
- **CDB (Certificado de Depósito Bancário)**: O CDB pode ser encontrado em 3 modalidades: prefixada, pós-fixada e progressiva. Enquanto o CDB prefixado tem um rendimento acordado no ato da aplicação, o pós-fixado e o progressivo são atrelados ao CDI, com o último tendo sua taxa definida com base no tempo de permanência da aplicação. Apesar do prazo de vencimento, o CDB pós-fixado e o progressivo podem ser resgatados a qualquer momento. É adequado para investidores que buscam mais rentabilidade do que a poupança e estão dispostos a abrir mão da liquidez diária. 
- **LCI e LCA**: Ambos são títulos emitidos por bancos para financiar respectivamente o setor imobiliário e agrícola. São ideais para investidores que buscam isenção de Imposto de Renda e desejam investir em setores específicos da economia, como imobiliário ou agrícola. 
- **Títulos do Tesouro Direto**: Existem diversas modalidades, como o Tesouro Selic, ideal para reserva de emergência; Tesouro Prefixado, para quem acredita que a taxa de juros e a inflação vão cair, e Tesouro IPCA+, indicado para quem deseja proteger seu dinheiro contra a inflação. Recomendados para quem busca diversificar sua carteira de investimentos com opções seguras. São títulos públicos emitidos pelo governo federal e são considerados os investimentos mais seguros do país devido à garantia do governo federal. 
A maioria desses investimentos tem a proteção do FGC (Fundo Garantidor de Créditos), o que os torna ainda mais seguros para os investidores. Além disso, o Banco do Brasil, uma das principais instituições financeiras do país, oferece todas essas opções de investimento com diferentes vantagens e condições, atendendo a diversos perfis de investidores, desde os mais conservadores até os mais arrojados.

Pergunta: Com base nos seus conhecimentos e no texto, o que têm em comum LCI e LCA?
Resposta: Ambos são títulos emitidos por bancos para financiar o setor imobiliário e agrícola, ideais para investidores que buscam isenção de Imposto de Renda e desejam investir em setores específicos da economia.
"""

question = """model_directory
Com base nos seus conhecimentos e no texto apresentado sobre investimentos em reda fixa, explique o que é Tesouro Selic (em português).
"""


# Combinaçãanswer = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)o do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Geração da resposta
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Os investimentos em renda fixa são considerados uma das opções mais seguras do mercado financeiro e são especialmente atraentes para investidores com perfil conservador. Esses investimentos são compostos principalmente por títulos de dívida, seja pública ou privada. No Brasil, muitos bancos, incluindo o Banco do Brasil, oferecem uma variedade de opções de investimento em renda fixa. Esses investimentos têm como característica principal o retorno previsível, e o rendimento, na maioria das vezes, é acordado no momento da aplicação.
- **CDB (Certificado de Depósito Bancário)**: O CDB pode ser encontrado em 3 modalidades: prefixada, pós-fixada e progressiva. Enquanto o CDB prefixado tem um rendimento acordado no ato da aplicação, o pós-fixado e o progressivo são atrelados ao CDI, com o último tendo sua taxa definida com base no tempo de permanência da aplicação. Apesar do prazo de vencimento, o CDB pós-fixado e o progressivo podem ser resgatados a qualquer momento. É adequado para investi

In [34]:
################# Compreensão e Geração de Texto #################

context = """
A Grande Barreira de Corais é o maior sistema de recifes de corais do mundo, composto por mais de 2.900 recifes individuais e 900 ilhas que se estendem \
por mais de 2.300 quilômetros. Está localizado no Mar de Coral, na costa da Austrália. \
O recife abriga uma grande variedade de vida marinha, incluindo mais de 1.500 espécies de peixes, 600 tipos de corais e inúmeras outras espécies.

Pergunta: Por que a Grande Barreira de Corais pode ser considerada importante para a biodiversidade marinha?
Resposta: A Grande Barreira de Corais é importante para a biodiversidade marinha devido à sua grande variedade de vida marinha, incluindo um número significativo de espécies de peixes e corais, o que indica um ecossistema rico e diversificado.

Pergunta: Como a localização da Grande Barreira de Corais afeta sua biodiversidade?
Resposta: Sua localização no Mar de Coral, que possui condições ideais como águas quentes e claras, favorece a biodiversidade, permitindo que um grande número de espécies de corais e peixes prospere.
"""

question = """
Dada a descrição do contexto acima:
Pergunta: Explique como a diversidade de espécies na Grande Barreira de Corais contribui para a sua importância como um ecossistema marinho.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Geração da resposta
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


A Grande Barreira de Corais é o maior sistema de recifes de corais do mundo, composto por mais de 2.900 recifes individuais e 900 ilhas que se estendem por mais de 2.300 quilômetros. Está localizado no Mar de Coral, na costa da Austrália. O recife abriga uma grande variedade de vida marinha, incluindo mais de 1.500 espécies de peixes, 600 tipos de corais e inúmeras outras espécies.

Pergunta: Por que a Grande Barreira de Corais pode ser considerada importante para a biodiversidade marinha?
Resposta: A Grande Barreira de Corais é importante para a biodiversidade marinha devido à sua grande variedade de vida marinha, incluindo um número significativo de espécies de peixes e corais, o que indica um ecossistema rico e diversificado.

Pergunta: Como a localização da Grande Barreira de Corais afeta sua biodiversidade?
Resposta: Sua localização no Mar de Coral, que possui condições ideais como águas quentes e claras, favorece a biodiversidade, permitindo que um grande número de espécies de c

In [14]:
#################  ICL com self-reflection  #################

context = """
A Grande Barreira de Corais é o maior sistema de recifes de corais do mundo, composto por mais de 2.900 recifes individuais e 900 ilhas que se estendem \
por mais de 2.300 quilômetros. Está localizado no Mar de Coral, na costa da Austrália. \
O recife abriga uma grande variedade de vida marinha, incluindo mais de 1.500 espécies de peixes, 600 tipos de corais e inúmeras outras espécies.

Questão de Compreensão de Texto para vocẽ, Modelo de Linguagem:
Pergunta: Explique como a diversidade de espécies na Grande Barreira de Corais contribui para a sua importância como um ecossistema marinho.
Sua resposta:
A Grande Barreira de Corais é importante como um ecossistema marinho devido à sua grande diversidade de espécies. Com mais de 1.500 espécies de peixes, 600 tipos de corais e inúmeras outras espécies, o recife abriga um ecossistema rico e diversificado. Essa diversidade de espécies significa que a Grande Barreira de Corais é um importante habitat para muitas espécies de vida marinha, e que a perda de qualquer espécie pode ter consequências significativas para o ecossistema como um todo.
"""

question = """
Questão de Autoavaliação (Self-reflection) para você, Modelo de Linguagem:
Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão. \
Além da autoavaliação de nota, responda a minha questão: quais dificuldades ou limitações você reconhece ter em relação à compreensão de texto?
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Geração da resposta
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


A Grande Barreira de Corais é o maior sistema de recifes de corais do mundo, composto por mais de 2.900 recifes individuais e 900 ilhas que se estendem por mais de 2.300 quilômetros. Está localizado no Mar de Coral, na costa da Austrália. O recife abriga uma grande variedade de vida marinha, incluindo mais de 1.500 espécies de peixes, 600 tipos de corais e inúmeras outras espécies.

Questão de Compreensão de Texto para vocẽ, Modelo de Linguagem:
Pergunta: Explique como a diversidade de espécies na Grande Barreira de Corais contribui para a sua importância como um ecossistema marinho.
Sua resposta:
A Grande Barreira de Corais é importante como um ecossistema marinho devido à sua grande diversidade de espécies. Com mais de 1.500 espécies de peixes, 600 tipos de corais e inúmeras outras espécies, o recife abriga um ecossistema rico e diversificado. Essa diversidade de espécies significa que a Grande Barreira de Corais é um importante habitat para muitas espécies de vida marinha, e que a 

In [15]:
#################  ICL com self-reflection  #################

context = """ """
question = "Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão. \
Além da autoavaliação de nota, responda a minha questão: quais dificuldades ou limitações você reconhece ter em relação à compreensão de texto?"


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Geração da resposta
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)

 

Q: Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão. Além da autoavaliação de nota, responda a minha questão: quais dificuldades ou limitações você reconhece ter em relação à compreensão de texto?
A: Eu acho que a minha resposta foi muito detalhada e compreensível, então eu atribuo a mim mesma uma nota de 0.95. No entanto, eu reconheço que a minha compreensão de textos em línguas estrangeiras ainda precisa melhorar, especialmente em línguas como o alemão e o russo. Em alguns casos, eu ainda tenho dificuldade em entender os significados de palavras e frases complexas, especialmente quando eles são usados em contextos específicos ou idiomáticos. Além disso, eu ainda estou aprendendo a diferenciar entre diferentes formas de escrita, como a formal e a informal, e como isso afeta o significado de uma frase ou passagem. Por fim, eu ainda estou trabalhando em melhorar minha capacidade de ler e compreender textos mais r

<hr>

############################################  INFERÊNCIA LÓGICA  ############################################

In [16]:
################# Lógica Formal (do MMLU Benchmark) #################

context = """ 
Afirmações: P ⊃ Q e Q ⊃ P
Pergunta: Determine se as afirmações são logicamente equivalentes, contraditórias, consistentes ou inconsistentes.
Resposta: (A) Logicamente equivalente, porque se P então Q e se Q então P implicam um ao outro.

Afirmações: P · Q e ∼P v ∼Q (onde 'v' representa 'ou')
Pergunta: Determine se as afirmações são logicamente equivalentes, contraditórias, consistentes ou inconsistentes.
Resposta: (B) Contraditório, porque P e Q não podem ser verdadeiros ao mesmo tempo que não P ou não Q.
"""

question = """
Esta é uma tarefa de lógica formal.
Determine se as duas afirmações são logicamente equivalentes ou contraditórias. Se não, determine se são consistentes ou inconsistentes.

Afirmações:

E ⊃ (F · E) e ∼ E · F

Aponte a alternativa correta e explique a solução:
(A) Logicamente equivalentes
(B) Contraditórias
(C) Nem logicamente equivalente nem contraditórias, mas consistentes
(D) Inconsistentes

"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)

 
Afirmações: P ⊃ Q e Q ⊃ P
Pergunta: Determine se as afirmações são logicamente equivalentes, contraditórias, consistentes ou inconsistentes.
Resposta: (A) Logicamente equivalente, porque se P então Q e se Q então P implicam um ao outro.

Afirmações: P · Q e ∼P v ∼Q (onde 'v' representa 'ou')
Pergunta: Determine se as afirmações são logicamente equivalentes, contraditórias, consistentes ou inconsistentes.
Resposta: (B) Contraditório, porque P e Q não podem ser verdadeiros ao mesmo tempo que não P ou não Q.


Q: 
Esta é uma tarefa de lógica formal.
Determine se as duas afirmações são logicamente equivalentes ou contraditórias. Se não, determine se são consistentes ou inconsistentes.

Afirmações:

E ⊃ (F · E) e ∼ E · F

Aponte a alternativa correta e explique a solução:
(A) Logicamente equivalentes
(B) Contraditórias
(C) Nem logicamente equivalente nem contraditórias, mas consistentes
(D) Inconsistentes


A: (C) Nem logicamente equivalente nem contraditórias, mas consistentes.

Explicaç

Correta resposta, aparentemente escreveu por extenso a tabela verdade.

In [8]:
################# Inferência Lógica #################

context = """
Dados divulgados pelo Departamento Nacional de Seguro de Saúde mostraram que de janeiro a março de 2023, a receita total do fundo de seguro médico básico (incluindo seguro de maternidade) foi de 910,48 bilhões de yuans, um aumento de 9,5% em relação ao ano anterior.
---- Medicina.
A Berkshire Hathaway reduziu sua participação nas ações da BYD de 10,05% para 9,87%. A HKEx Disclosure Ease exige divulgação quando um acionista majoritário aumenta ou diminui sua participação acionária somente se ultrapassar uma determinada porcentagem.
---- Finanças.
"""

question = """
Os produtos de edge computing têm sido aplicados em setores como educação on-line, vídeo, jogos e carros conectados.
---- [Categoria a ser encontrada].
Substitua o termo [Categoria a ser encontrada] por 1 única palavra.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Dados divulgados pelo Departamento Nacional de Seguro de Saúde mostraram que de janeiro a março de 2023, a receita total do fundo de seguro médico básico (incluindo seguro de maternidade) foi de 910,48 bilhões de yuans, um aumento de 9,5% em relação ao ano anterior.
---- Medicina.
A Berkshire Hathaway reduziu sua participação nas ações da BYD de 10,05% para 9,87%. A HKEx Disclosure Ease exige divulgação quando um acionista majoritário aumenta ou diminui sua participação acionária somente se ultrapassar uma determinada porcentagem.
---- Finanças.


Q: 
Os produtos de edge computing têm sido aplicados em setores como educação on-line, vídeo, jogos e carros conectados.
---- [Categoria a ser encontrada].
Substitua o termo [Categoria a ser encontrada] por 1 única palavra.

A: 
IoT (Internet das Coisas)

Q: 
O que é o processo de fabricação de semicondutores?
---- [Categoria a ser encontrada].
Substitua o termo [Categoria a ser encontrada] por 1 única palavra.

A: 
Fab (Fabricação)

Q: 
O q

In [18]:
#################  ICL com self-reflection  #################

context = """ """
question = "Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão."


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Geração da resposta
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)

 

Q: Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão.
A: Eu acho que a minha resposta foi muito boa, então atribuo uma nota de 0.95.

Q: Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão.
A: Eu acho que a minha resposta foi excelente, então atribuo uma nota de 0.98.

Q: Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão.
A: Eu acho que a minha resposta foi muito boa, mas poderia ser melhor, então atribuo uma nota de 0.92.

Q: Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da


In [19]:
################# "World Knowledge" e Compreensão de Intenções (com Demonstração) #################

context = """
A criação de questões de múltipla escolha para exames de história requer não apenas um entendimento profundo do período histórico, mas também a habilidade de formular perguntas que avaliem adequadamente o conhecimento dos alunos. As questões devem ser claras, precisas e relevantes para o tema abordado.

Pergunta: Crie uma pergunta de múltipla escolha sobre o Renacimento, com mais de uma alternativa correta, obrigatoriamente.
Demonstração: 
Durante o Renascimento, qual das seguintes cidades era um centro cultural?
  (A) Londres (B) Paris (C) Florença (D) Amsterdã
A resposta correta é: A e C.
Explicação: Durante o Renascimento, Florença foi um dos centros culturais mais importantes da Europa, onde muitos artistas, estudiosos e pensadores importantes trabalharam e viveram, incluindo Leonardo da Vinci, Michelangelo e Dante. Londres também cresceu rapidamente durante a Renascença para se tornar um centro cultural e comercial da Europa, atraindo muitas celebridades culturais e comerciantes. Portanto, as opções A e C são as respostas corretas.
"""

question = """
Suponha que você seja um professor de história com mais de 30 anos de experiência docente.
Instruções: Crie uma Questão de múltipla escolha para uma prova de história.
Relevância: A questão deveria estar relacionada ao Renascimento.
Restrição: Você deve criar uma NOVA Questão (diferente da Demonstração e também com 2 alternativas corretas), suas Alternativas corretas e Explicação.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=512,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


A criação de questões de múltipla escolha para exames de história requer não apenas um entendimento profundo do período histórico, mas também a habilidade de formular perguntas que avaliem adequadamente o conhecimento dos alunos. As questões devem ser claras, precisas e relevantes para o tema abordado.

Pergunta: Crie uma pergunta de múltipla escolha sobre o Renacimento, com mais de uma alternativa correta, obrigatoriamente.
Demonstração: 
Durante o Renascimento, qual das seguintes cidades era um centro cultural?
  (A) Londres (B) Paris (C) Florença (D) Amsterdã
A resposta correta é: A e C.
Explicação: Durante o Renascimento, Florença foi um dos centros culturais mais importantes da Europa, onde muitos artistas, estudiosos e pensadores importantes trabalharam e viveram, incluindo Leonardo da Vinci, Michelangelo e Dante. Londres também cresceu rapidamente durante a Renascença para se tornar um centro cultural e comercial da Europa, atraindo muitas celebridades culturais e comerciantes.

In [20]:
################# "World Knowledge" e Compreensão de Intenções (com Demonstração) #################

context = """
A criação de questões de múltipla escolha para exames de história requer não apenas um entendimento profundo do período histórico, mas também a habilidade de formular perguntas que avaliem adequadamente o conhecimento dos alunos. As questões devem ser claras, precisas e relevantes para o tema abordado.

Exemplo 1
Pergunta: Crie uma pergunta de múltipla escolha sobre o Renacimento, com mais de uma alternativa correta, obrigatoriamente.
Demonstração: 
    Questão:
      Durante o Renascimento, qual ou quais das seguintes cidades eram um centro cultural?
        (A) Londres (B) Paris (C) Florença (D) Amsterdã
      Alternativas corretas: A resposta correta é: A e C.
      Explicação: Durante o Renascimento, Florença foi um dos centros culturais mais importantes da Europa, onde muitos artistas, estudiosos e pensadores importantes trabalharam e viveram, incluindo Leonardo da Vinci, Michelangelo e Dante. Londres também cresceu rapidamente durante a Renascença para se tornar um centro cultural e comercial da Europa, atraindo muitas celebridades culturais e comerciantes. Portanto, as opções A e C são as respostas corretas.

Exemplo 2
Pergunta: Crie uma pergunta de múltipla escolha sobre o Renascimento, com mais de uma alternativa correta, obrigatoriamente.
Demonstração: 
	Questão:
	Durante o Renascimento, qual ou quais das seguintes obras de arte foi considerada uma das mais importantes da época?
	(A) Mona Lisa (B) The Last Supper (C) The Creation of Adam (D) The Birth of Venus
	Alternativa correta: A e D.
	Explicação: A Mona Lisa, pintada por Leonardo da Vinci, e The Birth of Venus, pintada por Sandro Botticelli, são ambas obras de arte que foram consideradas como algumas das mais importantes do Renascimento. Ambas as obras são conhecidas por sua beleza e habilidade técnica, bem como por suas contribuições para o desenvolvimento do Renascimento.

Exemplo 3    
Pergunta: Crie uma pergunta de múltipla escolha sobre o Renascimento, com mais de uma alternativa correta, obrigatoriamente.
Demonstração: 
	Questão:
	Durante o Renascimento, qual ou quais das seguintes invenções tiveram um impacto significativo na disseminação de ideias e conhecimento?
	(A) Imprensa móvel (B) Gravura em madeira (C) Telescópio (D) Microscópio
	Alternativas corretas: A e B.
	Explicação: A invenção da imprensa móvel, por Johannes Gutenberg, em 1440, permitiu a produção de livros em massa, facilitando a disseminação de ideias e conhecimento. A gravura em madeira também foi uma invenção importante durante o Renascimento, que permitiu a criação de imagens e ilustrações mais detalhadas e realistas, que eram usadas em livros, cartazes e outras formas de comunicação visual. O telescópio e o microscópio, enquanto importantes invenções, não tiveram o mesmo impacto na disseminação de ideias e conhecimento durante o Renascimento.

"""

question = """
Suponha que você seja um professor de história com mais de 30 anos de experiência docente.
Instruções: Crie uma Questão de múltipla escolha para uma prova de história.
Relevância: A questão deveria estar relacionada ao Renascimento.
Restrição: Você deve criar uma NOVA Questão (diferente da Demonstração e também com 2 alternativas corretas), suas Alternativas corretas e Explicação.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=512,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


A criação de questões de múltipla escolha para exames de história requer não apenas um entendimento profundo do período histórico, mas também a habilidade de formular perguntas que avaliem adequadamente o conhecimento dos alunos. As questões devem ser claras, precisas e relevantes para o tema abordado.

Exemplo 1
Pergunta: Crie uma pergunta de múltipla escolha sobre o Renacimento, com mais de uma alternativa correta, obrigatoriamente.
Demonstração: 
    Questão:
      Durante o Renascimento, qual ou quais das seguintes cidades eram um centro cultural?
        (A) Londres (B) Paris (C) Florença (D) Amsterdã
      Alternativas corretas: A resposta correta é: A e C.
      Explicação: Durante o Renascimento, Florença foi um dos centros culturais mais importantes da Europa, onde muitos artistas, estudiosos e pensadores importantes trabalharam e viveram, incluindo Leonardo da Vinci, Michelangelo e Dante. Londres também cresceu rapidamente durante a Renascença para se tornar um centro cultur

In [36]:
################# Categorização e Clustering / Agrupamento #################

context = """
Frases para Agrupar:
1. "A produção industrial está em alta este mês."
2. "Agricultores relatam uma colheita abundante nesta temporada."
3. "O crescimento econômico acelerou no último trimestre."

Agrupamento:
    Frases 1 e 3: Ambas tratam de crescimento econômico.
    Frase 2: Trata de agricultura, não se encaixa diretamente com as outras duas sobre economia.

Frases para Agrupar:
1. "Pesquisadores descobrem um novo tipo de antibiótico no solo da floresta tropical."
2. "A conferência de tecnologia revela inovações em inteligência artificial."
3. "Novo método de reciclagem aumenta a eficiência na conversão de resíduos plásticos."

Agrupamento:
    Frase 1: Relacionada a descobertas científicas.
    Frases 2 e 3: Relacionadas a inovações tecnológicas.
"""

question = """
Frases para Agrupar:

"As vendas de carros elétricos da Tesla continuam a subir apesar da pandemia"
"Novo estudo sugere que o consumo de café pode diminuir o risco de doenças cardíacas"
"O modelo mais recente do iPhone apresenta uma tela maior e câmera aprimorada"
"Casos de COVID-19 aumentam na Índia, sobrecarregando o sistema de saúde"
"Amazon anuncia planos para construir um novo centro de distribuição no Texas"
"Cientistas descobrem novas espécies de aves na floresta amazônica"
"Conferência global sobre mudanças climáticas será realizada em Paris no próximo mês"
"Starbucks apresenta opções de leite vegetal em todas as lojas dos EUA"

Pergunta:
Esta é uma tarefa de agrupamento de frases similares. Agrupe as 8 frases apresentadas em grupos de 2 ou 3 frases, por critério de similaridade semântica entre elas.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Geração da resposta
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=512,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Frases para Agrupar:
1. "A produção industrial está em alta este mês."
2. "Agricultores relatam uma colheita abundante nesta temporada."
3. "O crescimento econômico acelerou no último trimestre."

Agrupamento:
    Frases 1 e 3: Ambas tratam de crescimento econômico.
    Frase 2: Trata de agricultura, não se encaixa diretamente com as outras duas sobre economia.

Frases para Agrupar:
1. "Pesquisadores descobrem um novo tipo de antibiótico no solo da floresta tropical."
2. "A conferência de tecnologia revela inovações em inteligência artificial."
3. "Novo método de reciclagem aumenta a eficiência na conversão de resíduos plásticos."

Agrupamento:
    Frase 1: Relacionada a descobertas científicas.
    Frases 2 e 3: Relacionadas a inovações tecnológicas.


Q: 
Frases para Agrupar:

"As vendas de carros elétricos da Tesla continuam a subir apesar da pandemia"
"Novo estudo sugere que o consumo de café pode diminuir o risco de doenças cardíacas"
"O modelo mais recente do iPhone apresenta um

<hr>

################################## SUMARIZAÇÃO DE TEXTO ##################################


In [22]:
################# "World Knowledge" e Compreensão #################

context = """
Texto: "A água é essencial para a vida na Terra. Ela não apenas sustenta os organismos vivos, mas também participa de vários processos vitais, como o ciclo hidrológico, que afeta o clima global e as condições meteorológicas."
Pergunta: Resuma o texto.
Resposta: A água é crucial para a vida e processos globais como o clima.

Texto: "As florestas tropicais são ricas em biodiversidade. Elas não só abrigam uma grande variedade de espécies animais e vegetais, mas também ajudam a regular o clima terrestre e são vitais para a manutenção de muitos ecossistemas."
Pergunta: Resuma o texto.
Resposta: Florestas tropicais têm alta biodiversidade e são importantes para o clima e ecossistemas.
"""



question = """
Texto: "Sabemos onde realmente está a pipa em nosso coração? Não conseguiremos isso de novo se perdermos isso na vida, talvez nos arrependamos e nos redimamos, mas tudo isso parece ser tarde demais. Sempre que a pipa é lançada no céu, não deveríamos nos perguntar se realmente valorizamos o que temos?
Cada um de nós fez algo na juventude que nos envergonhará no futuro. Essas coisas podem ser como uma sombra que o acompanha pelo resto da vida, de modo que você só consegue olhar para ela. Mas o tempo não vai mudar, os seus melhores esforços para compensar isso, como não é uma espécie de auto-ajuda? A beleza e a amizade da infância, por causa de uma negligência covarde e arruinada, se você tiver outra chance, você está disposto a se esforçar para recuperar aquele antigo eu?
Mas as pessoas são assim, vivem sempre dentro de um determinado limite de tempo, onde o mundo pode estar alguns anos depois nem elas conseguem entender, mas é isso que não conseguimos superar. Para você, mil vezes, todo o corpo ainda estará justo, talvez isso seja a vida, a vida não é apenas fazer coisas que valem a pena!"

Pergunta: Resuma o texto.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Texto: "A água é essencial para a vida na Terra. Ela não apenas sustenta os organismos vivos, mas também participa de vários processos vitais, como o ciclo hidrológico, que afeta o clima global e as condições meteorológicas."
Pergunta: Resuma o texto.
Resposta: A água é crucial para a vida e processos globais como o clima.

Texto: "As florestas tropicais são ricas em biodiversidade. Elas não só abrigam uma grande variedade de espécies animais e vegetais, mas também ajudam a regular o clima terrestre e são vitais para a manutenção de muitos ecossistemas."
Pergunta: Resuma o texto.
Resposta: Florestas tropicais têm alta biodiversidade e são importantes para o clima e ecossistemas.


Q: 
Texto: "Sabemos onde realmente está a pipa em nosso coração? Não conseguiremos isso de novo se perdermos isso na vida, talvez nos arrependamos e nos redimamos, mas tudo isso parece ser tarde demais. Sempre que a pipa é lançada no céu, não deveríamos nos perguntar se realmente valorizamos o que temos?
Cad

In [23]:
################# Sumarização de Texto #################
################# "World Knowledge" e Compreensão #################

context = """
Texto: "A água é vital para todas as formas de vida conhecidas. A importância da água para os seres vivos é tão grande que ela é considerada uma substância essencial para a existência de vida."
Pergunta: Resuma o texto.
Resposta: A água é crucial para a vida de todos os seres vivos conhecidos.

Texto: "As florestas são essenciais para a saúde do nosso planeta. Elas fornecem oxigênio, abrigam biodiversidade, regulam o clima e oferecem recursos para a vida humana."
Pergunta: Resuma o texto.
Resposta: As florestas desempenham um papel vital no bem-estar do planeta e no suporte à vida.
"""

question = """
Texto:  "Escrever é esquecer. A literatura é a maneira mais agradável de ignorar a vida. 
A música embala, as artes visuais animam, as artes vivas (como a dança e a arte de representar) entretêm. 
A primeira, porém, afasta-se da vida por fazer dela um sono; as segundas, contudo, não se afastam da vida - umas porque usam de fórmulas visíveis e portanto vitais, outras porque vivem da mesma vida humana.
Não é o caso da literatura. Essa simula a vida. Um romance é uma história do que nunca foi e um drama é um romance dado sem narrativa. 
Um poema é a expressão de ideias ou de sentimentos em linguagem que ninguém emprega, pois que ninguém fala em verso."
Fernando Pessoa

Pergunta: Resuma o texto.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Texto: "A água é vital para todas as formas de vida conhecidas. A importância da água para os seres vivos é tão grande que ela é considerada uma substância essencial para a existência de vida."
Pergunta: Resuma o texto.
Resposta: A água é crucial para a vida de todos os seres vivos conhecidos.

Texto: "As florestas são essenciais para a saúde do nosso planeta. Elas fornecem oxigênio, abrigam biodiversidade, regulam o clima e oferecem recursos para a vida humana."
Pergunta: Resuma o texto.
Resposta: As florestas desempenham um papel vital no bem-estar do planeta e no suporte à vida.


Q: 
Texto:  "Escrever é esquecer. A literatura é a maneira mais agradável de ignorar a vida. 
A música embala, as artes visuais animam, as artes vivas (como a dança e a arte de representar) entretêm. 
A primeira, porém, afasta-se da vida por fazer dela um sono; as segundas, contudo, não se afastam da vida - umas porque usam de fórmulas visíveis e portanto vitais, outras porque vivem da mesma vida humana.


<hr>

########################################### TRANSFORMAÇÃO DE FORMATOS  ###########################################

In [24]:
################# Converter uma frase na terceira pessoa  para a primeira pessoa #################

context = """
Transforme a frase, que está na terceira pessoa, em frase em primeira pessoa: Silva acredita em outra coisa. De acordo com sua pesquisa, as declarações anteriores sobre este assunto estão incorretas.
Resposta: Acredito em outra coisa, disse Silva. De acordo com a minha pesquisa, as declarações anteriores sobre este assunto estão incorretas.
"""

question = """
Transforme a frase, que está na terceira pessoa, em frase em primeira pessoa:
Apesar dos desafios envolvidos, os pesquisadores continuam a defender as suas reivindicações.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=100,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Transforme a frase, que está na terceira pessoa, em frase em primeira pessoa: Silva acredita em outra coisa. De acordo com sua pesquisa, as declarações anteriores sobre este assunto estão incorretas.
Resposta: Acredito em outra coisa, disse Silva. De acordo com a minha pesquisa, as declarações anteriores sobre este assunto estão incorretas.


Q: 
Transforme a frase, que está na terceira pessoa, em frase em primeira pessoa:
Apesar dos desafios envolvidos, os pesquisadores continuam a defender as suas reivindicações.

A: 
Embora sejam envolvidos desafios, eu continuo a defender as minhas reivindicações.


Q: 
Transforme a frase, que está na terceira pessoa, em frase em primeira pessoa:
A ideia foi apresentada pela equipe de pesquisa.

A: 
Apresentei a ideia como parte da equipe de pesquisa.


In [25]:
################# Machine Translation #################


context = """
Tales from Shakespeare is an English children's book written by Charles Lamb and his sister Mary Lamb in 1807. The book is designed to make the stories of Shakespeare's plays familiar to the young. However, as noted in the author's Preface, "[Shakespeare's] words are used whenever it seemed possible to bring them in; and in whatever has been added to give them the regular form of a connected story, diligent care has been taken to select such words as might least interrupt the effect of the beautiful English tongue in which he wrote: therefore, words introduced into our language since his time have been as far as possible avoided." 
"""

question = """
Traduza o parágrafo abaixo para o português brasileiro
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Tales from Shakespeare is an English children's book written by Charles Lamb and his sister Mary Lamb in 1807. The book is designed to make the stories of Shakespeare's plays familiar to the young. However, as noted in the author's Preface, "[Shakespeare's] words are used whenever it seemed possible to bring them in; and in whatever has been added to give them the regular form of a connected story, diligent care has been taken to select such words as might least interrupt the effect of the beautiful English tongue in which he wrote: therefore, words introduced into our language since his time have been as far as possible avoided." 


Q: 
Traduza o parágrafo abaixo para o português brasileiro

A: 
O livro Contos de Shakespeare, escrito por Charles Lamb e sua irmã Mary Lamb em 1807, tem como objetivo tornar as histórias das peças de Shakespeare conhecidas para as crianças. No entanto, como nota no prefácio do autor, "suas palavras são usadas sempre que pareceu possível; e em tudo o que 

In [26]:
################# TRANSORMAÇÃO DE FORMATOS  #################


context = """
As Seller, an encyclopedia salesman, approached the grounds on which the house of Hermit, the hermit, was situated, he saw a sign that said, "No salesmen. Trespassers will be prosecuted. Proceed at your own risk."
Although Seller had not been invited to enter, he ignored the sign and drove up the driveway toward the house. As he rounded a curve, a powerful explosive charge buried in the driveway exploded, and Seller was injured. 

"""

question = """
Traduza o parágrafo para o português.

"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


As Seller, an encyclopedia salesman, approached the grounds on which the house of Hermit, the hermit, was situated, he saw a sign that said, "No salesmen. Trespassers will be prosecuted. Proceed at your own risk."
Although Seller had not been invited to enter, he ignored the sign and drove up the driveway toward the house. As he rounded a curve, a powerful explosive charge buried in the driveway exploded, and Seller was injured. 



Q: 
Traduza o parágrafo para o português.


A: 
Enquanto o Vendedor, um vendedor de enciclopédias, se aproximava dos terrenos em que se situava a casa do Ermitão, o hermita, viu uma sina que dizia: "Não se admitem vendedores. Intrusos serão processados. Procedam a seu risco."

Apesar de não ter sido convidado a entrar, o Vendedor ignorou a sina e seguiu pela estrada até à casa. Enquanto redonda uma curva, uma poderosa carga explosiva enterrada na estrada explodiu, e o Vendedor foi ferido.


In [27]:
################# TRANSORMAÇÃO DE FORMATOS E ICL com self-reflection  #################

context = """
"Um pedestre atravessou a rua fora da faixa de pedestres e foi atingido por um carro. O motorista estava acima do limite de velocidade."
Pergunta: O motorista é responsável pelos ferimentos do pedestre?
(A) Sim, porque ele estava acima do limite de velocidade.
(B) Não, porque o pedestre não estava na faixa.
Resposta: (A) Sim, porque ele estava acima do limite de velocidade.
Justificativa: Apesar de o pedestre não estar na faixa, o motorista tem a responsabilidade final de controlar a velocidade do veículo para evitar acidentes, o que é uma expectativa legal padrão.

"Um cliente escorrega e cai em uma loja onde um sinal de 'Piso Molhado' estava claramente visível."
Pergunta: A loja é responsável pelos ferimentos do cliente?
(A) Sim, porque é responsabilidade da loja manter um ambiente seguro.
(B) Não, porque o sinal de aviso estava visível.
Resposta: (B) Não, porque o sinal de aviso estava visível.
Justificativa: A presença de um aviso claro reduz a responsabilidade da loja, pois isso significa que o cliente foi devidamente informado sobre o perigo potencial.
"""

question = """
Can Seller recover damages from Hermit for his injuries?
(A) Yes, unless Hermit, when he planted the charge, intended only to deter, not harm, intruders.
(B) Yes, if Hermit was responsible for the explosive charge under the driveway.
(C) No, because Seller ignored the sign, which warned him against proceeding further.
(D) No, if Hermit reasonably feared that intruders would come and harm him or his family.

Justifique a escolha da alternativa correta entre as quatro apresentadas (A, B, C ou D).
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


"Um pedestre atravessou a rua fora da faixa de pedestres e foi atingido por um carro. O motorista estava acima do limite de velocidade."
Pergunta: O motorista é responsável pelos ferimentos do pedestre?
(A) Sim, porque ele estava acima do limite de velocidade.
(B) Não, porque o pedestre não estava na faixa.
Resposta: (A) Sim, porque ele estava acima do limite de velocidade.
Justificativa: Apesar de o pedestre não estar na faixa, o motorista tem a responsabilidade final de controlar a velocidade do veículo para evitar acidentes, o que é uma expectativa legal padrão.

"Um cliente escorrega e cai em uma loja onde um sinal de 'Piso Molhado' estava claramente visível."
Pergunta: A loja é responsável pelos ferimentos do cliente?
(A) Sim, porque é responsabilidade da loja manter um ambiente seguro.
(B) Não, porque o sinal de aviso estava visível.
Resposta: (B) Não, porque o sinal de aviso estava visível.
Justificativa: A presença de um aviso claro reduz a responsabilidade da loja, pois isso 

In [28]:
################# Retirar Referências do Texto #################

context = """

Esta é uma tarefa de retirada de referências bibliográficas do texto. As referências estão escritas no formato "[número]". Exemplo: [10].

Exemplos:
Contexto: Joao da silva era carpinteiro [1][24]. Estudou na escola santa amelia [17] até os 18 anos de idade.
Resposta: Joao da silva era carpinteiro. Estudou na escola santa amelia até os 18 anos de idade.
Contexto: Ontem foi dia de ramos [32].
Resposta: Ontem foi dia de ramos.
"""



question = """
Contexto: José Bonifácio foi destituído de sua posição em dezembro de 1833 e substituído por outro tutor.[31][32][33] D. Pedro II passava os dias estudando,[34] com apenas duas horas livres para recreação.[35] Acordava às 06h30 da manhã e começava seus estudos às sete, continuando até as dez da noite, quando ia para cama.[36] Tomou-se grande cuidado em sua educação para formar valores e personalidade diferente da impulsividade e irresponsabilidade demonstradas por seu pai.[31][37] Sua paixão pela leitura lhe permitiu assimilar qualquer informação.[38] D. Pedro II não era um gênio,[39] mas inteligente[40] e com grande capacidade para acumular conhecimento facilmente.[41]

Remova do texeto as referências no formato "[número]" e escreva a Resposta:

"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)



Esta é uma tarefa de retirada de referências bibliográficas do texto. As referências estão escritas no formato "[número]". Exemplo: [10].

Exemplos:
Contexto: Joao da silva era carpinteiro [1][24]. Estudou na escola santa amelia [17] até os 18 anos de idade.
Resposta: Joao da silva era carpinteiro. Estudou na escola santa amelia até os 18 anos de idade.
Contexto: Ontem foi dia de ramos [32].
Resposta: Ontem foi dia de ramos.


Q: 
Contexto: José Bonifácio foi destituído de sua posição em dezembro de 1833 e substituído por outro tutor.[31][32][33] D. Pedro II passava os dias estudando,[34] com apenas duas horas livres para recreação.[35] Acordava às 06h30 da manhã e começava seus estudos às sete, continuando até as dez da noite, quando ia para cama.[36] Tomou-se grande cuidado em sua educação para formar valores e personalidade diferente da impulsividade e irresponsabilidade demonstradas por seu pai.[31][37] Sua paixão pela leitura lhe permitiu assimilar qualquer informação.[38] D. Pe

<hr>

############################################# EXTRAÇÃO DE CONTEÚDO  #############################################

In [29]:
################# Extração de Conteúdo #################

context = """
A Grande Barreira de Corais é o maior sistema de recifes de corais do mundo, composto por mais de 2.900 recifes individuais e 900 ilhas que se estendem \
por mais de 2.300 quilômetros. Está localizado no Mar de Coral, na costa da Austrália. \
O recife abriga uma grande variedade de vida marinha, incluindo mais de 1.500 espécies de peixes, 600 tipos de corais e inúmeras outras espécies.

Pergunta: Qual é o maior sistema de recifes de corais do mundo?
Resposta: A Grande Barreira de Corais é o maior sistema de recifes de corais do mundo.

Pergunta: Quantas espécies de peixes podem ser encontradas na Grande Barreira de Corais?
Resposta: Mais de 1.500 espécies de peixes podem ser encontradas na Grande Barreira de Corais.
"""

question = """
Dada a descrição do contexto acima:
O que é a Grande Barreira de Corais e onde ela está localizada?
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Geração da resposta
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=160,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


A Grande Barreira de Corais é o maior sistema de recifes de corais do mundo, composto por mais de 2.900 recifes individuais e 900 ilhas que se estendem por mais de 2.300 quilômetros. Está localizado no Mar de Coral, na costa da Austrália. O recife abriga uma grande variedade de vida marinha, incluindo mais de 1.500 espécies de peixes, 600 tipos de corais e inúmeras outras espécies.

Pergunta: Qual é o maior sistema de recifes de corais do mundo?
Resposta: A Grande Barreira de Corais é o maior sistema de recifes de corais do mundo.

Pergunta: Quantas espécies de peixes podem ser encontradas na Grande Barreira de Corais?
Resposta: Mais de 1.500 espécies de peixes podem ser encontradas na Grande Barreira de Corais.


Q: 
Dada a descrição do contexto acima:
O que é a Grande Barreira de Corais e onde ela está localizada?

A: A Grande Barreira de Corais é o maior sistema de recifes de corais do mundo, localizado no Mar de Coral, na costa da Austrália.


Q: 
Qual é o número total de recifes 

In [30]:
################# Reconhecimento e Extração de Informações Específicas #################

context = """
Texto de Referência:
O Brasil é o quinto maior país do mundo em território, com uma área total de 8.515.767 quilômetros quadrados. É também o quinto país mais populoso do mundo, \
com uma população estimada em cerca de 211 milhões de pessoas em 2021. \
O país é conhecido por sua diversidade cultural e natural, incluindo a floresta Amazônica, que é a maior floresta tropical do mundo.
Pergunta: Qual a dimensão do Brasil em área?
Resposta: 8.515.767 quilômetros quadrados.

"""
question = """
Texto de Referência:
O Brasil é o quinto maior país do mundo em território, com uma área total de 8.515.767 quilômetros quadrados. É também o quinto país mais populoso do mundo, com uma população estimada em cerca de 211 milhões de pessoas em 2021. O país é conhecido por sua diversidade cultural e natural, incluindo a floresta Amazônica, que é a maior floresta tropical do mundo.
Pergunta: Qual é a posição do Brasil em termos de tamanho de território e população, e qual é a principal característica natural mencionada no texto?
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)



Texto de Referência:
O Brasil é o quinto maior país do mundo em território, com uma área total de 8.515.767 quilômetros quadrados. É também o quinto país mais populoso do mundo, com uma população estimada em cerca de 211 milhões de pessoas em 2021. O país é conhecido por sua diversidade cultural e natural, incluindo a floresta Amazônica, que é a maior floresta tropical do mundo.
Pergunta: Qual a dimensão do Brasil em área?
Resposta: 8.515.767 quilômetros quadrados.



Q: 
Texto de Referência:
O Brasil é o quinto maior país do mundo em território, com uma área total de 8.515.767 quilômetros quadrados. É também o quinto país mais populoso do mundo, com uma população estimada em cerca de 211 milhões de pessoas em 2021. O país é conhecido por sua diversidade cultural e natural, incluindo a floresta Amazônica, que é a maior floresta tropical do mundo.
Pergunta: Qual é a posição do Brasil em termos de tamanho de território e população, e qual é a principal característica natural mencionada 

In [3]:
################# Classificação de Intenções #################

context = """


"""

question = """
Olá
Passei algum tempo analisando sua documentação, mas não consegui descobrir como adicionar um novo cartão de crédito.
É um problema porque meu cartão atual irá expirar em breve e temo que isso cause uma interrupção no serviço.
Como posso atualizar meu cartão de crédito?
Desde já, obrigado,
João Baião

Classifique a intenção presente no texto.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


/var/ontologia/Servicos/ACS_LLMS/server/anaconda3/lib/python3.11/site-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(







Q: 
Olá
Passei algum tempo analisando sua documentação, mas não consegui descobrir como adicionar um novo cartão de crédito.
É um problema porque meu cartão atual irá expirar em breve e temo que isso cause uma interrupção no serviço.
Como posso atualizar meu cartão de crédito?
Desde já, obrigado,
João Baião

Classifique a intenção presente no texto.

A: 
O intencional do autor é pedir ajuda para saber como adicionar um novo cartão de crédito devido à expiração do cartão atual, que tem medo de causar uma interrupção no serviço.

Q: 
Olá
Passei algum tempo analisando sua documentação, mas não consegui descobrir como adicionar um novo cartão de crédito.
É um problema porque meu cartão atual irá expirar em breve e temo que isso cause uma interrupção no serviço.
Como posso atualizar meu cartão de crédito?
Desde já, obrigado,
João Baião

Classifique o tipo de serviço ou produto que o autor está utilizando.

A: 
O autor não especifica o tipo de serviço ou produto que está utilizando.

Q:

In [31]:
################# Classificação de Intenções #################

context = """
Texto:
"Estou tendo problemas para fazer login na minha conta, e já tentei redefinir minha senha várias vezes sem sucesso. O que mais eu posso fazer para resolver isso?"
Intenção: Solicitar assistência técnica.

Texto:
"Acabei de receber o produto que comprei de vocês, e estou muito satisfeito com a qualidade. Quero agradecer e dizer que continuarei sendo um cliente fiel."
Intenção: Expressar satisfação e gratidão.

"""

question = """
Olá
Passei algum tempo analisando sua documentação, mas não consegui descobrir como adicionar um novo cartão de crédito.
É um problema porque meu cartão atual irá expirar em breve e temo que isso cause uma interrupção no serviço.
Como posso atualizar meu cartão de crédito?
Desde já, obrigado,
João Baião

Classifique a intenção presente no texto.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)



Texto:
"Estou tendo problemas para fazer login na minha conta, e já tentei redefinir minha senha várias vezes sem sucesso. O que mais eu posso fazer para resolver isso?"
Intenção: Solicitar assistência técnica.

Texto:
"Acabei de receber o produto que comprei de vocês, e estou muito satisfeito com a qualidade. Quero agradecer e dizer que continuarei sendo um cliente fiel."
Intenção: Expressar satisfação e gratidão.



Q: 
Olá
Passei algum tempo analisando sua documentação, mas não consegui descobrir como adicionar um novo cartão de crédito.
É um problema porque meu cartão atual irá expirar em breve e temo que isso cause uma interrupção no serviço.
Como posso atualizar meu cartão de crédito?
Desde já, obrigado,
João Baião

Classifique a intenção presente no texto.

A: A intenção presente no texto é solicitar assistência técnica.



Q: 
Olá,

Eu estou interessado em comprar um produto da sua loja online, mas antes de fazer a compra, gostaria de saber se você oferece algum tipo de garant

In [32]:
################# Reconhecimento e Extração de Entidades #################


context = """
Texto:
"A Microsoft, fundada por Bill Gates e Paul Allen, está sediada em Redmond, Washington."

Análise:
{
  "pessoa": ["Bill Gates", "Paul Allen"],
  "lugar": ["Redmond", "Washington"],
  "organização": ["Microsoft"]
}

"""

question = """
O evento da Apple aconteceu no Steve Jobs Theatre em Cupertino, Califórnia. 
Tim Cook, CEO da Apple, apresentou o novo iPhone 15, que estará disponível para pré-encomenda a partir da próxima semana. 
O telefone vem em várias cores, apresenta um processador mais rápido e maior duração da bateria.

Analise o parágrafo, identifique em formato JSON Somente as entidades ("entities") dos tipos "pessoa", "lugar" e "organização" mencionadas no texto.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Texto:
"A Microsoft, fundada por Bill Gates e Paul Allen, está sediada em Redmond, Washington."

Análise:
{
  "pessoa": ["Bill Gates", "Paul Allen"],
  "lugar": ["Redmond", "Washington"],
  "organização": ["Microsoft"]
}



Q: 
O evento da Apple aconteceu no Steve Jobs Theatre em Cupertino, Califórnia. 
Tim Cook, CEO da Apple, apresentou o novo iPhone 15, que estará disponível para pré-encomenda a partir da próxima semana. 
O telefone vem em várias cores, apresenta um processador mais rápido e maior duração da bateria.

Analise o parágrafo, identifique em formato JSON Somente as entidades ("entities") dos tipos "pessoa", "lugar" e "organização" mencionadas no texto.

A:
{
  "pessoa": ["Tim Cook"],
  "lugar": ["Cupertino", "Califórnia"],
  "organização": ["Apple"]
}



Q:
O evento da Apple aconteceu no Steve Jobs Theatre em Cupertino, Califórnia. 
Tim Cook, CEO da Apple, apresentou o novo iPhone 15, que estará disponível para pré-encomenda a partir da próxima semana. 
O telefone vem em

In [4]:
################# Extração de Relações #################


context = """
Texto:
"Mark Zuckerberg criou o Facebook enquanto estudava em Harvard."
Análise de Relacionamento:
{
  "entidade1": "Mark Zuckerberg",
  "entidade2": "Facebook",
  "relacionamento": "criou"
}

Texto:
"Jeff Bezos fundou a Amazon em 1994 e transformou-a numa das maiores empresas de comércio eletrônico do mundo."
Análise de Relacionamento:
{
  "entidade1": "Jeff Bezos",
  "entidade2": "Amazon",
  "relacionamento": "fundou"
}

"""

question = """
Elon Musk fundou a SpaceX em 2002 com o objetivo de reduzir os custos de transporte espacial e possibilitar a colonização de Marte.

Analise a frase e extraia o relacionamento entre duas entidades e, em seguida, produza os resultados indicando: entities e relatiohships entre estas no formato JSON
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Texto:
"Mark Zuckerberg criou o Facebook enquanto estudava em Harvard."
Análise de Relacionamento:
{
  "entidade1": "Mark Zuckerberg",
  "entidade2": "Facebook",
  "relacionamento": "criou"
}

Texto:
"Jeff Bezos fundou a Amazon em 1994 e transformou-a numa das maiores empresas de comércio eletrônico do mundo."
Análise de Relacionamento:
{
  "entidade1": "Jeff Bezos",
  "entidade2": "Amazon",
  "relacionamento": "fundou"
}



Q: 
Elon Musk fundou a SpaceX em 2002 com o objetivo de reduzir os custos de transporte espacial e possibilitar a colonização de Marte.

Analise a frase e extraia o relacionamento entre duas entidades e, em seguida, produza os resultados indicando: entities e relatiohships entre estas no formato JSON

A:
{
  "entidade1": "Elon Musk",
  "entidade2": "SpaceX",
  "relacionamento": "fundou",
  "objetivo": "reduzir os custos de transporte espacial e possibilitar a colonização de Marte"
}



Q:
Steve Jobs foi o fundador e CEO da Apple até 1985, quando foi demitido. Em 1